In [120]:
import pandas as pd
import numpy as np

In [121]:
#Importar dados:
url_commodities = "https://raw.githubusercontent.com/edufsi/Projeto-Desafio/main/Dados/UNdata_Export_20240412_201846138.csv"
url_parceiros = "https://raw.githubusercontent.com/edufsi/Projeto-Desafio/main/Dados/SYB66_330_202310_Major%20Trading%20Partners.csv"
url_exportacoes = "https://raw.githubusercontent.com/edufsi/Projeto-Desafio/main/Dados/SYB66_123_202310_Total%20Imports%20Exports%20and%20Balance%20of%20Trade.csv"
url_cambio = "https://raw.githubusercontent.com/edufsi/Projeto-Desafio/main/Dados/USD_BRL_Historical_Data.csv"


dados_commodities = pd.read_csv(url_commodities)
dados_parceiros = pd.read_csv(url_parceiros, encoding = "ISO-8859-1")
dados_exportacoes = pd.read_csv(url_exportacoes, encoding = "ISO-8859-1")
dados_cambio = pd.read_csv(url_cambio)

# Dados de Commodities


### Limpeza dos dados

In [122]:
# Visao geral dos dados de commodities
print(dados_commodities.head())

print("=================================")

dados_commodities.info()
dados_commodities.describe()

# Como da pra ver, as colunas Weight (kg) e Quantity nao possuem nenhum valor nao nulo, vamos retira-las

# Retirando colunas Weight (kg) e Quantity
dados_commodities.drop(['Quantity', 'Weight (kg)'], axis=1, inplace=True)


  Country or Area  Year        Commodity       Flow   Trade (USD)  \
0     Afghanistan  2019  All Commodities     Export  8.704885e+08   
1     Afghanistan  2019  All Commodities     Import  8.568014e+09   
2     Afghanistan  2019  All Commodities  Re-Export  6.655197e+06   
3     Afghanistan  2018  All Commodities     Import  7.406590e+09   
4     Afghanistan  2018  All Commodities  Re-Export  9.263097e+06   

   Weight (kg) Quantity Name  Quantity  
0          NaN   No Quantity       NaN  
1          NaN   No Quantity       NaN  
2          NaN   No Quantity       NaN  
3          NaN   No Quantity       NaN  
4          NaN   No Quantity       NaN  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11180 entries, 0 to 11179
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Country or Area  11180 non-null  object 
 1   Year             11180 non-null  int64  
 2   Commodity        11180 non-null  object 
 

In [123]:
mascara = dados_commodities['Quantity Name'] != "No Quantity"

print(dados_commodities[mascara]) # nenhuma linha apresenta algo diferente de "No Quantity" na coluna Qunatity Name, o que faz sentido, já que não tínhamos quantidade alguma

# Vamos tirar essa coluna também
dados_commodities.drop('Quantity Name', axis=1, inplace=True)


Empty DataFrame
Columns: [Country or Area, Year, Commodity, Flow, Trade (USD), Quantity Name]
Index: []


In [124]:
# Nos livramos de 3 colunas inúteis até agora. Vamos olhar para a coluna Commoditiy, que tambem parece suspeita:

mascara = dados_commodities.query('Commodity != "All Commodities"')
print(mascara)

# Novamente, não existem dois valores diferentes para esta coluna. Ela só informa que estamos lidando com a importação e exportação total das
# commodities da região especificada. Por isso, não há motivo para a coluna estar ali
dados_commodities.drop('Commodity', axis=1, inplace=True)


Empty DataFrame
Columns: [Country or Area, Year, Commodity, Flow, Trade (USD)]
Index: []


In [156]:
# Vamos ver de qual ano ate qual ano nossos dados vao
print(dados_commodities['Year'].min()) # 1988
print(dados_commodities['Year'].max()) # 2022


# Vamos ver quais paises fizeram mais importacoes e exportacoes de commodities em 2022 e quais fizeram menos

commodities_2022 = dados_commodities.query('Year == 2022')

exportacoes_2022 = commodities_2022.query('Flow == "Export"')
importacoes_2022 = commodities_2022.query('Flow == "Import"')

mascara = exportacoes_2022['Trade (USD)'] == exportacoes_2022['Trade (USD)'].max() # Mais exportacoes: China
print(exportacoes_2022[mascara])

mascara = exportacoes_2022['Trade (USD)'] == exportacoes_2022['Trade (USD)'].min() # Menos exportacoes: Monserrate (Ilha caribenha)
print(exportacoes_2022[mascara])

mascara = importacoes_2022['Trade (USD)'] == importacoes_2022['Trade (USD)'].max() # Mais importacoes: EUA
print(importacoes_2022[mascara])

mascara = importacoes_2022['Trade (USD)'] == importacoes_2022['Trade (USD)'].min() # Menos importacoes: Monserrate (Ilha caribenha)
print(importacoes_2022[mascara])


1988
2022
     Country or Area  Year    Flow   Trade (USD)
2114           China  2022  Export  3.593601e+12
     Country or Area  Year    Flow  Trade (USD)
6609      Montserrat  2022  Export  6645359.367
      Country or Area  Year    Flow   Trade (USD)
10773             USA  2022  Import  3.372902e+12
     Country or Area  Year    Flow   Trade (USD)
6608      Montserrat  2022  Import  3.923705e+07


### Entendendo os Dados

In [126]:
# Finalamente temos so dados uteis. Nao temos nenhum NaN nos dados tambem.

# Entendo a estrutura do dataframe
print(dados_commodities.tail(50))

# No geral, temos as importações, exportações e reexportações de commodities (em USD) de países durante um ano. Para alguns paises
# algumas dessas informaçoes faltam e os anos tambem nao estao sincronizados


''' O que é reexportação?
O termo reexportação refere-se ao facto de um país, território, região ou cidade importar produtos e exportá-los em seguida. Geralmente os produtos
reexportados não sofrem qualquer tipo de processo para agregar valor (industrialização), ou seja, são vendidos como foram comprados. A finalidade da
reexportação é exportar produtos a um valor superior ao custo de importação e assim acumular capitais para seu território. Poucos países no mundo
utilizam-se dessa prática, principalmente pequenos países com capacidade produtiva limitada '''

      Country or Area  Year       Flow   Trade (USD)
11130        Zimbabwe  2015  Re-Export  2.305893e+07
11131        Zimbabwe  2015     Import  6.052863e+09
11132        Zimbabwe  2015     Export  3.410659e+09
11133        Zimbabwe  2014  Re-Export  5.565467e+07
11134        Zimbabwe  2014     Import  6.069848e+09
11135        Zimbabwe  2014     Export  3.871574e+09
11136        Zimbabwe  2013  Re-Export  9.232217e+07
11137        Zimbabwe  2013     Import  7.216365e+09
11138        Zimbabwe  2013     Export  4.067665e+09
11139        Zimbabwe  2012  Re-Export  4.535589e+07
11140        Zimbabwe  2012     Export  4.125176e+09
11141        Zimbabwe  2012     Import  6.938769e+09
11142        Zimbabwe  2011  Re-Export  4.098900e+07
11143        Zimbabwe  2011     Import  7.984010e+09
11144        Zimbabwe  2011     Export  3.121209e+09
11145        Zimbabwe  2010     Export  2.640578e+09
11146        Zimbabwe  2010  Re-Export  4.124002e+07
11147        Zimbabwe  2010     Import  5.3373

' O que é reexportação?\nO termo reexportação refere-se ao facto de um país, território, região ou cidade importar produtos e exportá-los em seguida. Geralmente os produtos\nreexportados não sofrem qualquer tipo de processo para agregar valor (industrialização), ou seja, são vendidos como foram comprados. A finalidade da\nreexportação é exportar produtos a um valor superior ao custo de importação e assim acumular capitais para seu território. Poucos países no mundo\nutilizam-se dessa prática, principalmente pequenos países com capacidade produtiva limitada '

### Analises

In [127]:
# Vamos começar a analise agrupando os dados por paises e flow, tirando a totalidade das importações, exportações e rexportações em todos os anos

totalidade_trocas = dados_commodities.groupby(['Country or Area', 'Flow'])['Trade (USD)'].sum() # Agrupando por pais e por flow e tirando a soma das importaçoes, exportaçoes e reexportaçoes

print(totalidade_trocas)

Country or Area  Flow     
ASEAN            Export       1.345040e+13
                 Import       1.309573e+13
Afghanistan      Export       6.977031e+09
                 Import       7.838164e+10
                 Re-Export    1.591829e+07
                                  ...     
Zambia           Import       1.233642e+11
                 Re-Export    7.138532e+09
Zimbabwe         Export       7.776759e+10
                 Import       1.098411e+11
                 Re-Export    8.791804e+08
Name: Trade (USD), Length: 534, dtype: float64


In [128]:
# Uma informação interessante é a diferença entre importações, exportações e reexportações

# Como nao temos uma padronizacao, nao podemos diretamente fazer algo como:
  # totalidade_trocas.values.reshape(-1, 3) para tomar exatamente os valores de importações, exportações e reexportações



# Para evitar ter que adicionar valores que padronizem a series resultante, podemos simplesmente utilizar o método unstack()

# unstack(): toma um dataframe multiindexado e tranforma o nivel passado como pivo em colunas contendo seus valores.
# No exemplo que estamos usando, unstack pegará cada tipo de troca (importação, exportação e reexportação), que estão agrupados
# por países, e irá transformar eles em colunas, cada linha representando um pais:

print(totalidade_trocas.to_frame().unstack())


# Se quiséssemos tranformar cada país em coluna com cada linha representando um tipo de troca, poderiamos usar unstack passando 0 como
# parametro, ou seja, o primeiro nivel do dataframe multiindexado. Como padrão, unstack toma o último nível (-1)

print(totalidade_trocas.to_frame().unstack(0))

                         Trade (USD)                            
Flow                          Export        Import     Re-Export
Country or Area                                                 
ASEAN                   1.345040e+13  1.309573e+13           NaN
Afghanistan             6.977031e+09  7.838164e+10  1.591829e+07
Albania                 4.012638e+10  1.039977e+11  1.826849e+09
Algeria                 9.172711e+11  6.823703e+11           NaN
Andorra                 3.097698e+09  3.889995e+10           NaN
...                              ...           ...           ...
Viet Nam                2.578649e+12  2.625487e+12           NaN
Wallis and Futuna Isds           NaN  3.096408e+08           NaN
Yemen                   6.701891e+10  1.141707e+11  2.685559e+09
Zambia                  1.305524e+11  1.233642e+11  7.138532e+09
Zimbabwe                7.776759e+10  1.098411e+11  8.791804e+08

[212 rows x 3 columns]
                  Trade (USD)                                     

In [145]:
df_trocas = totalidade_trocas.to_frame().unstack()

# Temos agora que lidar com os NaNs, que surgiram pois havia paises sem algumas informaçoes. Vamos transformar todos em 0

df_trocas.fillna(0, inplace=True)






# Finalmente, basta fazer as diferencas das colunas
balanco = df_trocas['Trade (USD)']['Export'] + df_trocas['Trade (USD)']['Re-Export'] - df_trocas['Trade (USD)']['Import']




# E uma medida. Nao necessariamente muito boa, visto que paises diferentes tem quantidade de anos diferentes comparados. Mas ja é alguma medida
# Podemos tirar, por exemplo, o pais com o balanco mais positivo com relacao a commodities e o com o mais negativo:

balanco.index # agora, balanco é uma series cujos indices sao os nomes dos paises. Podemos usar metodos para tratar series, ou transforma-la de volta em df usando reset_index

balanco = balanco.reset_index()

balanco.columns = ['Pais ou Area', 'Valor']


# MINIMO:
print(balanco[balanco['Valor'] == balanco['Valor'].min()]) # EUA

# MAXIMO:
print(balanco[balanco['Valor'] == balanco['Valor'].max()]) # China


print(balanco)


                  Trade (USD)                        
Flow                   Export        Import Re-Export
Country or Area                                      
EU-28            4.169654e+13  4.393599e+13       0.0
                       Trade (USD)                       
Flow                        Export       Import Re-Export
Country or Area                                          
Chad                           0.0  215168272.0       0.0
Wallis and Futuna Isds         0.0  309640836.0       0.0
    Pais ou Area         Valor
197          USA -1.575545e+13
   Pais ou Area         Valor
41        China  6.892394e+12
               Pais ou Area         Valor
0                     ASEAN  3.546680e+11
1               Afghanistan -7.138869e+10
2                   Albania -6.204448e+10
3                   Algeria  2.349008e+11
4                   Andorra -3.580226e+10
..                      ...           ...
207                Viet Nam -4.683748e+10
208  Wallis and Futuna Isds -3.09640

In [130]:
# Em vez da diferença absoluta, podemos ver tambem a diferenca relativa. Isso pode nos dar mais informaçoes sobre paises menores
# Vamos ver a diferenca da exportacao com a importacao em porcentagem (quanto por cento do valor da exportacao as importacoes representam):

dif_rel_commodities = df_trocas['Trade (USD)']['Import'] / (df_trocas['Trade (USD)']['Export'] + df_trocas['Trade (USD)']['Re-Export'])

dif_rel_commodities = dif_rel_commodities.reset_index()
dif_rel_commodities.columns = ['Pais ou Area', 'Importacao/Exportacao']




# MINIMO:
print(dif_rel_commodities[dif_rel_commodities['Importacao/Exportacao'] == dif_rel_commodities['Importacao/Exportacao'].min()]) # Haiti
  # No Haiti nao ha registro de importacoes. Se tirassemos ele, o menor seria:

dif_rel_commodities = dif_rel_commodities[dif_rel_commodities['Pais ou Area'] != 'Haiti']
print(dif_rel_commodities[dif_rel_commodities['Importacao/Exportacao'] == dif_rel_commodities['Importacao/Exportacao'].min()]) # Iraque



# MAXIMO:
print(dif_rel_commodities[dif_rel_commodities['Importacao/Exportacao'] == dif_rel_commodities['Importacao/Exportacao'].max()]) # Chade (infinito, nao ha registro de exportacao)

   Pais ou Area  Importacao/Exportacao
88        Haiti                    0.0
   Pais ou Area  Importacao/Exportacao
95         Iraq               0.074501
               Pais ou Area  Importacao/Exportacao
39                     Chad                    inf
208  Wallis and Futuna Isds                    inf


# Dados de Exportações

### Limpeza

In [131]:
print(dados_exportacoes.head())

# Os titulos das colunas estao errados. Eles estao na segunda coluna. Na primeira temos um titulo


                   T21 Total imports, exports and balance of trade Unnamed: 2  \
0  Region/Country/Area                                         NaN       Year   
1                    1               Total, all countries or areas       1995   
2                    1               Total, all countries or areas       2005   
3                    1               Total, all countries or areas       2010   
4                    1               Total, all countries or areas       2015   

                             Unnamed: 3       Unnamed: 4  \
0                                Series  System of trade   
1  Imports CIF (millions of US dollars)              NaN   
2  Imports CIF (millions of US dollars)              NaN   
3  Imports CIF (millions of US dollars)              NaN   
4  Imports CIF (millions of US dollars)              NaN   

                 Unnamed: 5  Unnamed: 6 Unnamed: 7  \
0  System of trade footnote       Value  Footnotes   
1                       NaN   5,099,057     

In [132]:
print(dados_exportacoes.columns)

Index(['T21', 'Total imports, exports and balance of trade', 'Unnamed: 2',
       'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7',
       'Unnamed: 8'],
      dtype='object')


In [133]:
# Podemos usar iloc para selecionar linhas. Passando um intervalo, uma lista ou um numero retornamos as linhas correspondentes

# Vamos tomar a primeira linha e defini-la como nossas colunas
dados_exportacoes.columns = dados_exportacoes.iloc[0]


# Agora que os titulos ja estao corretos, podemos excluir essa primeira linha

dados_exportacoes.drop(0, inplace=True)

dados_exportacoes.head()


,Region/Country/Area,NaN,Year,Series,System of trade,System of trade footnote,Value,Footnotes,Source
1,1,"Total, all countries or areas",1995,Imports CIF (millions of US dollars),NaN,NaN,"5,099,057",NaN,"United Nations Statistics Division, New York, ..."
2,1,"Total, all countries or areas",2005,Imports CIF (millions of US dollars),NaN,NaN,"10,577,013",NaN,"United Nations Statistics Division, New York, ..."
3,1,"Total, all countries or areas",2010,Imports CIF (millions of US dollars),NaN,NaN,"15,275,115",NaN,"United Nations Statistics Division, New York, ..."
4,1,"Total, all countries or areas",2015,Imports CIF (millions of US dollars),NaN,NaN,"16,482,346",NaN,"United Nations Statistics Division, New York, ..."
5,1,"Total, all countries or areas",2020,Imports CIF (millions of US dollars),NaN,NaN,"17,598,001",NaN,"United Nations Statistics Division, New York, ..."


In [134]:
# Ainda há uma coluna "errada" no entanto. No arquivo original, foram associados valores para representar as regioes/paises/areas e há uma coluna sem titulo para a descricao
# Seria melhor aqui ter apenas uma coluna com os nomes das regiões, áreas e dos países. Temos que alter entao o nome da segunda coluna, essa sem titulo


lista = [x for x in dados_exportacoes.columns]
lista[1] = "Region/Country" # Substituindo o nome da coluna com nan por "Region/Country" (segunda coluna)

dados_exportacoes.columns = lista

print(dados_exportacoes.columns)




Index(['Region/Country/Area', 'Region/Country', 'Year', 'Series',
       'System of trade', 'System of trade footnote', 'Value', 'Footnotes',
       'Source'],
      dtype='object')


In [135]:
# Finalmente os titulos estao corretos
print(dados_exportacoes.head())

  Region/Country/Area                 Region/Country  Year  \
1                   1  Total, all countries or areas  1995   
2                   1  Total, all countries or areas  2005   
3                   1  Total, all countries or areas  2010   
4                   1  Total, all countries or areas  2015   
5                   1  Total, all countries or areas  2020   

                                 Series System of trade  \
1  Imports CIF (millions of US dollars)             NaN   
2  Imports CIF (millions of US dollars)             NaN   
3  Imports CIF (millions of US dollars)             NaN   
4  Imports CIF (millions of US dollars)             NaN   
5  Imports CIF (millions of US dollars)             NaN   

  System of trade footnote       Value Footnotes  \
1                      NaN   5,099,057       NaN   
2                      NaN  10,577,013       NaN   
3                      NaN  15,275,115       NaN   
4                      NaN  16,482,346       NaN   
5           

In [136]:
# Informacoes dos dados

print(dados_exportacoes.info())

# Podemos retirar a coluna "System of trade footnote, pois ela nao tem nenhum valor nao nulo"
# Tambem podemos tirar a coluna de codificacao das regioes, ja que iremos nos referir a elas pelo nome

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5053 entries, 1 to 5053
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Region/Country/Area       5053 non-null   object
 1   Region/Country            5053 non-null   object
 2   Year                      5053 non-null   object
 3   Series                    5053 non-null   object
 4   System of trade           4381 non-null   object
 5   System of trade footnote  0 non-null      object
 6   Value                     5053 non-null   object
 7   Footnotes                 1523 non-null   object
 8   Source                    5053 non-null   object
dtypes: object(9)
memory usage: 355.4+ KB
None


In [137]:
dados_exportacoes.drop(["System of trade footnote", "Region/Country/Area"], axis=1, inplace=True)


In [138]:
dados_exportacoes

,Region/Country,Year,Series,System of trade,Value,Footnotes,Source
1,"Total, all countries or areas",1995,Imports CIF (millions of US dollars),NaN,"5,099,057",NaN,"United Nations Statistics Division, New York, ..."
2,"Total, all countries or areas",2005,Imports CIF (millions of US dollars),NaN,"10,577,013",NaN,"United Nations Statistics Division, New York, ..."
3,"Total, all countries or areas",2010,Imports CIF (millions of US dollars),NaN,"15,275,115",NaN,"United Nations Statistics Division, New York, ..."
4,"Total, all countries or areas",2015,Imports CIF (millions of US dollars),NaN,"16,482,346",NaN,"United Nations Statistics Division, New York, ..."
5,"Total, all countries or areas",2020,Imports CIF (millions of US dollars),NaN,"17,598,001",NaN,"United Nations Statistics Division, New York, ..."
...,...,...,...,...,...,...,...
5049,Zimbabwe,2010,Balance imports/exports (millions of US dollars),G,"-2,653",NaN,"United Nations Statistics Division, New York, ..."
5050,Zimbabwe,2015,Balance imports/exports (millions of US dollars),G,"-2,642",NaN,"United Nations Statistics Division, New York, ..."
5051,Zimbabwe,2020,Balance imports/exports (millions of US dollars),G,-653,NaN,"United Nations Statistics Division, New York, ..."
5052,Zimbabwe,2021,Balance imports/exports (millions of US dollars),G,"-1,541",NaN,"United Nations Statistics Division, New York, ..."


In [139]:
# A coluna de valores esta como tipo objeto. Precisamos tranformá-la em numero. Nao temos numeros quebrados, podemos transformá-la em numeros inteiros
# Precisamos retirar as virgulas dos numeros, para entao transforma-los em inteiros


dados_exportacoes['Value'] = dados_exportacoes['Value'].apply(lambda x: x.replace(',', '')).astype(np.int64)


print(dados_exportacoes.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5053 entries, 1 to 5053
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Region/Country   5053 non-null   object
 1   Year             5053 non-null   object
 2   Series           5053 non-null   object
 3   System of trade  4381 non-null   object
 4   Value            5053 non-null   int64 
 5   Footnotes        1523 non-null   object
 6   Source           5053 non-null   object
dtypes: int64(1), object(6)
memory usage: 276.5+ KB
None


In [140]:
dados_exportacoes.rename(columns={'System of trade': 'Trade_System'}, inplace=True)

dados_exportacoes




,Region/Country,Year,Series,Trade_System,Value,Footnotes,Source
1,"Total, all countries or areas",1995,Imports CIF (millions of US dollars),NaN,5099057,NaN,"United Nations Statistics Division, New York, ..."
2,"Total, all countries or areas",2005,Imports CIF (millions of US dollars),NaN,10577013,NaN,"United Nations Statistics Division, New York, ..."
3,"Total, all countries or areas",2010,Imports CIF (millions of US dollars),NaN,15275115,NaN,"United Nations Statistics Division, New York, ..."
4,"Total, all countries or areas",2015,Imports CIF (millions of US dollars),NaN,16482346,NaN,"United Nations Statistics Division, New York, ..."
5,"Total, all countries or areas",2020,Imports CIF (millions of US dollars),NaN,17598001,NaN,"United Nations Statistics Division, New York, ..."
...,...,...,...,...,...,...,...
5049,Zimbabwe,2010,Balance imports/exports (millions of US dollars),G,-2653,NaN,"United Nations Statistics Division, New York, ..."
5050,Zimbabwe,2015,Balance imports/exports (millions of US dollars),G,-2642,NaN,"United Nations Statistics Division, New York, ..."
5051,Zimbabwe,2020,Balance imports/exports (millions of US dollars),G,-653,NaN,"United Nations Statistics Division, New York, ..."
5052,Zimbabwe,2021,Balance imports/exports (millions of US dollars),G,-1541,NaN,"United Nations Statistics Division, New York, ..."


In [141]:
# vamos descartar tambem footnotes, que está servindo apenas para dizer onde uma informação é estimada
dados_exportacoes.drop('Footnotes', axis=1, inplace=True)

In [142]:
dados_exportacoes

,Region/Country,Year,Series,Trade_System,Value,Source
1,"Total, all countries or areas",1995,Imports CIF (millions of US dollars),NaN,5099057,"United Nations Statistics Division, New York, ..."
2,"Total, all countries or areas",2005,Imports CIF (millions of US dollars),NaN,10577013,"United Nations Statistics Division, New York, ..."
3,"Total, all countries or areas",2010,Imports CIF (millions of US dollars),NaN,15275115,"United Nations Statistics Division, New York, ..."
4,"Total, all countries or areas",2015,Imports CIF (millions of US dollars),NaN,16482346,"United Nations Statistics Division, New York, ..."
5,"Total, all countries or areas",2020,Imports CIF (millions of US dollars),NaN,17598001,"United Nations Statistics Division, New York, ..."
...,...,...,...,...,...,...
5049,Zimbabwe,2010,Balance imports/exports (millions of US dollars),G,-2653,"United Nations Statistics Division, New York, ..."
5050,Zimbabwe,2015,Balance imports/exports (millions of US dollars),G,-2642,"United Nations Statistics Division, New York, ..."
5051,Zimbabwe,2020,Balance imports/exports (millions of US dollars),G,-653,"United Nations Statistics Division, New York, ..."
5052,Zimbabwe,2021,Balance imports/exports (millions of US dollars),G,-1541,"United Nations Statistics Division, New York, ..."


In [143]:
# Finalmente temos um dataframe em que todas as colunas dizem algo util

,Region/Country,Year,Series,Trade_System,Value,Source
5004,Yemen,2022,Exports FOB (millions of US dollars),S,91,"United Nations Statistics Division, New York, ..."
5005,Yemen,1995,Balance imports/exports (millions of US dollars),S,105,"United Nations Statistics Division, New York, ..."
5006,Yemen,2005,Balance imports/exports (millions of US dollars),S,208,"United Nations Statistics Division, New York, ..."
5007,Yemen,2010,Balance imports/exports (millions of US dollars),S,-2818,"United Nations Statistics Division, New York, ..."
5008,Yemen,2015,Balance imports/exports (millions of US dollars),S,-6063,"United Nations Statistics Division, New York, ..."
5009,Yemen,2020,Balance imports/exports (millions of US dollars),S,-4632,"United Nations Statistics Division, New York, ..."
5010,Yemen,2021,Balance imports/exports (millions of US dollars),S,-5065,"United Nations Statistics Division, New York, ..."
5011,Yemen,2022,Balance imports/exports (millions of US dollars),S,-5184,"United Nations Statistics Division, New York, ..."
5012,Zambia,1995,Imports CIF (millions of US dollars),G,708,"United Nations Statistics Division, New York, ..."
5013,Zambia,2005,Imports CIF (millions of US dollars),G,2558,"United Nations Statistics Division, New York, ..."


### Análise

In [143]:
# Vamos olhar para o Brasil para fazer alguma analise
